In [1]:
import mysql.connector as mysql
import tkinter as tk
from tkinter import ttk, messagebox
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# Koneksi ke database MySQL
def connect_to_database():
    try:
        db = mysql.connect(
            host="localhost",
            user="root",
            passwd="root",
            database="tugasakhirdts_fix"
        )
        return db
    except mysql.Error as err:
        messagebox.showerror("Error", f"Error connecting to MySQL: {err}")
        return None

In [2]:
# Fungsi untuk menampilkan data
def view_data():
    db = connect_to_database()
    if db is None:
        return
    cursor = db.cursor()
    cursor.execute("SELECT * FROM HelenaComputerSchool")
    rows = cursor.fetchall()
    db.close()

    view_window = tk.Toplevel()
    view_window.title("Data Mahasiswa")

    tree = ttk.Treeview(view_window, columns=("ID_STUDENT", "DEGREE", "UNDERGRAD_GRADE", "WORK_EXP", "FINAL_GPA", "GRAD_PUNCTUAL"), show="headings")
    tree.heading("ID_STUDENT", text="ID")
    tree.heading("DEGREE", text="Degree")
    tree.heading("UNDERGRAD_GRADE", text="Grade")
    tree.heading("WORK_EXP", text="Work Exp")
    tree.heading("FINAL_GPA", text="GPA")
    tree.heading("GRAD_PUNCTUAL", text="Punctual")
    tree.pack(fill=tk.BOTH, expand=True)

    for row in rows:
        tree.insert("", tk.END, values=row)

In [14]:
def view_graduation_chart():
    db = connect_to_database()
    if db is None:
        return
    cursor = db.cursor()
    
    # Mengambil data dari database
    cursor.execute("SELECT FINAL_GPA, COUNT(*) FROM HelenaComputerSchool GROUP BY FINAL_GPA")
    data = cursor.fetchall()
    db.close()

    # Memproses data
    gpa_values = [row[0] for row in data]
    counts = [row[1] for row in data]

    # Membuat jendela grafik
    chart_window = tk.Toplevel()
    chart_window.title("Grafik Kelulusan")

    figure = Figure(figsize=(6, 4), dpi=100)
    plot = figure.add_subplot(1, 1, 1)
    plot.bar(gpa_values, counts)
    plot.set_title("Grafik Persebaran Berdasarkan Nilai Akhir (Final GPA)")
    plot.set_xlabel("Nilai Akhir (Final GPA)")
    plot.set_ylabel("Jumlah Mahasiswa")

    canvas = FigureCanvasTkAgg(figure, master=chart_window)
    canvas.draw()
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)


In [16]:
def view_graduation_pie_chart():
    db = connect_to_database()
    if db is None:
        return
    cursor = db.cursor()
    
    # Mengambil data dari database
    cursor.execute("""
        SELECT FINAL_GPA, COUNT(*)
        FROM HelenaComputerSchool
        WHERE GRAD_PUNCTUAL = 'Yes'
        GROUP BY FINAL_GPA
    """)
    data = cursor.fetchall()
    db.close()

    # Memproses data
    gpa_values = [row[0] for row in data]
    counts = [row[1] for row in data]

    # Membuat jendela pie chart
    pie_window = tk.Toplevel()
    pie_window.title("Pie Chart Kelulusan Tepat Waktu")

    figure = Figure(figsize=(6, 6), dpi=100)
    plot = figure.add_subplot(1, 1, 1)
    plot.pie(counts, labels=gpa_values, autopct='%1.1f%%', startangle=140)
    plot.set_title("Distribusi Kelulusan Tepat Waktu Berdasarkan Nilai Akhir (Final GPA)")

    canvas = FigureCanvasTkAgg(figure, master=pie_window)
    canvas.draw()
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

In [4]:
# Fungsi untuk menambah data
def add_data():
    def save_data():
        degree = degree_var.get()
        grade = int(grade_var.get())
        work_exp = work_exp_var.get()
        gpa = gpa_var.get()
        punctual = punctual_var.get()

        db = connect_to_database()
        if db is None:
            return
        cursor = db.cursor()
        try:
            cursor.execute(
                "INSERT INTO HelenaComputerSchool (DEGREE, UNDERGRAD_GRADE, WORK_EXP, FINAL_GPA, GRAD_PUNCTUAL) VALUES (%s, %s, %s, %s, %s)",
                (degree, grade, work_exp, gpa, punctual)
            )
            db.commit()
            messagebox.showinfo("Success", "Data berhasil ditambahkan!")
            add_window.destroy()
        except mysql.Error as err:
            messagebox.showerror("Error", f"Failed to insert data: {err}")
        finally:
            db.close()

    add_window = tk.Toplevel()
    add_window.title("Tambah Data Mahasiswa")

    tk.Label(add_window, text="Degree:").pack()
    degree_var = tk.StringVar()
    tk.Entry(add_window, textvariable=degree_var).pack()

    tk.Label(add_window, text="Grade:").pack()
    grade_var = tk.StringVar()
    tk.Entry(add_window, textvariable=grade_var).pack()

    tk.Label(add_window, text="Work Experience (Yes/No):").pack()
    work_exp_var = tk.StringVar()
    tk.Entry(add_window, textvariable=work_exp_var).pack()

    tk.Label(add_window, text="Final GPA:").pack()
    gpa_var = tk.StringVar()
    tk.Entry(add_window, textvariable=gpa_var).pack()

    tk.Label(add_window, text="Graduated Punctually (Yes/No):").pack()
    punctual_var = tk.StringVar()
    tk.Entry(add_window, textvariable=punctual_var).pack()

    tk.Button(add_window, text="Save", command=save_data).pack()

In [5]:
# Fungsi untuk menghapus data
def delete_data():
    def delete_selected():
        selected_item = tree.selection()
        if not selected_item:
            messagebox.showwarning("Warning", "No item selected")
            return

        item = tree.item(selected_item)
        id_student = item['values'][0]

        db = connect_to_database()
        if db is None:
            return
        cursor = db.cursor()
        try:
            cursor.execute("DELETE FROM HelenaComputerSchool WHERE ID_STUDENT = %s", (id_student,))
            db.commit()
            messagebox.showinfo("Success", "Data berhasil dihapus!")
            delete_window.destroy()
        except mysql.Error as err:
            messagebox.showerror("Error", f"Failed to delete data: {err}")
        finally:
            db.close()

    db = connect_to_database()
    if db is None:
        return
    cursor = db.cursor()
    cursor.execute("SELECT * FROM HelenaComputerSchool")
    rows = cursor.fetchall()
    db.close()

    delete_window = tk.Toplevel()
    delete_window.title("Hapus Data Mahasiswa")

    tree = ttk.Treeview(delete_window, columns=("ID_STUDENT", "DEGREE", "UNDERGRAD_GRADE", "WORK_EXP", "FINAL_GPA", "GRAD_PUNCTUAL"), show="headings")
    tree.heading("ID_STUDENT", text="ID")
    tree.heading("DEGREE", text="Degree")
    tree.heading("UNDERGRAD_GRADE", text="Grade")
    tree.heading("WORK_EXP", text="Work Exp")
    tree.heading("FINAL_GPA", text="GPA")
    tree.heading("GRAD_PUNCTUAL", text="Punctual")
    tree.pack(fill=tk.BOTH, expand=True)

    for row in rows:
        tree.insert("", tk.END, values=row)

    tk.Button(delete_window, text="Delete Selected", command=delete_selected).pack()

In [6]:
# Fungsi untuk mengedit data
def edit_data():
    def update_data():
        id_student = int(id_student_var.get())
        degree = degree_var.get()
        grade = int(grade_var.get())
        work_exp = work_exp_var.get()
        gpa = gpa_var.get()
        punctual = punctual_var.get()

        db = connect_to_database()
        if db is None:
            return
        cursor = db.cursor()
        try:
            cursor.execute(
                "UPDATE HelenaComputerSchool SET DEGREE=%s, UNDERGRAD_GRADE=%s, WORK_EXP=%s, FINAL_GPA=%s, GRAD_PUNCTUAL=%s WHERE ID_STUDENT=%s",
                (degree, grade, work_exp, gpa, punctual, id_student)
            )
            db.commit()
            messagebox.showinfo("Success", "Data berhasil diperbarui!")
            edit_window.destroy()
        except mysql.Error as err:
            messagebox.showerror("Error", f"Failed to update data: {err}")
        finally:
            db.close()

    edit_window = tk.Toplevel()
    edit_window.title("Edit Data Mahasiswa")

    tk.Label(edit_window, text="ID Student:").pack()
    id_student_var = tk.StringVar()
    tk.Entry(edit_window, textvariable=id_student_var).pack()

    tk.Label(edit_window, text="Degree:").pack()
    degree_var = tk.StringVar()
    tk.Entry(edit_window, textvariable=degree_var).pack()

    tk.Label(edit_window, text="Grade:").pack()
    grade_var = tk.StringVar()
    tk.Entry(edit_window, textvariable=grade_var).pack()

    tk.Label(edit_window, text="Work Experience (Yes/No):").pack()
    work_exp_var = tk.StringVar()
    tk.Entry(edit_window, textvariable=work_exp_var).pack()

    tk.Label(edit_window, text="Final GPA:").pack()
    gpa_var = tk.StringVar()
    tk.Entry(edit_window, textvariable=gpa_var).pack()

    tk.Label(edit_window, text="Graduated Punctually (Yes/No):").pack()
    punctual_var = tk.StringVar()
    tk.Entry(edit_window, textvariable=punctual_var).pack()

    tk.Button(edit_window, text="Update", command=update_data).pack()

In [17]:
import tkinter as tk
from PIL import Image, ImageTk  # Mengimpor Pillow untuk manipulasi gambar

def main_menu():
    root = tk.Tk()
    root.title("Aplikasi Manajemen Kelulusan Mahasiswa")
    root.geometry("700x400")  # Atur ukuran jendela
    root.configure(bg='lightblue')  # Mengatur background color menjadi biru muda

    # Menambahkan ikon di atas judul
    try:
        # Membuka gambar dan mengubah ukurannya
        image = Image.open("icon.png")
        image = image.resize((50, 50))  # Ubah ukuran ikon sesuai kebutuhan
        icon = ImageTk.PhotoImage(image)
    except (FileNotFoundError, IOError):
        icon = None
        print("Ikon tidak ditemukan")

    if icon:
        icon_label = tk.Label(root, image=icon, bg='lightblue')
        icon_label.photo = icon  # Menyimpan referensi ke gambar
        icon_label.pack(pady=(5, 2))  # Padding di atas dan bawah

    # Judul
    title_label = tk.Label(root, text="Selamat Datang di Aplikasi Manajemen Kelulusan Mahasiswa", font=("Arial", 16, "bold"), fg='black', bg='lightblue')
    title_label.pack(pady=10)

    tk.Button(root, text="Lihat Data Mahasiswa", command=view_data).pack(fill=tk.X, padx=20, pady=5)
    tk.Button(root, text="Lihat Grafik Nilai Akhir", command=view_graduation_chart).pack(fill=tk.X, padx=20, pady=5)
    tk.Button(root, text="Lihat Grafik Kelulusan", command=view_graduation_pie_chart).pack(fill=tk.X, padx=20, pady=5)
    tk.Button(root, text="Tambah Data Mahasiswa", command=add_data).pack(fill=tk.X, padx=20, pady=5)
    tk.Button(root, text="Hapus Data Mahasiswa", command=delete_data).pack(fill=tk.X, padx=20, pady=5)
    tk.Button(root, text="Edit Data Mahasiswa", command=edit_data).pack(fill=tk.X, padx=20, pady=5)

    root.mainloop()

if __name__ == "__main__":
    main_menu()


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/tkinter/__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/kj/cx6p9l8d0cs58zdd3pjmc6_r0000gn/T/ipykernel_20719/3578098715.py", line 60, in view_graduation_chart
    canvas = FigureCanvasTkAgg(figure, master=pie_window)
                                              ^^^^^^^^^^
NameError: name 'pie_window' is not defined
